Скачать набор данных creditcard.csv с https://www.kaggle.com/mlg-ulb/creditcardfraud, в котором: Time – время транзакции, V1-V28 – признаки, Amount – количество, Class – класс транзакции: мошенническая (Class=1) и легальная (Class=0).

Используя train_test_split(…, test_size=0.3, stratify=df.Class) разбейте выборку на тренировочное и тестовое множества так, чтобы тестовая выборка составляла 30% от всей выборки и в тестовой выборке пропорция мошеннических и легальных транзакций была такой же как и в основной выборке (stratify).

При решении задачи классификации транзакций на мошеннические и легальные необходимо максимизировать метрику macro f1 на тестовой выборке.

Задание 1. Применить методы бэггинга, случайного леса, LightGBM

Задание 2. Оптимизировать/настроить указанные (или любые другие) методы машинного обучения (не глубокого) для максимизации значения macro f1 на тестовой выборке. Для метода, обеспечившего максимум  macro f1, построить матрицу ошибок.

Задание 3. Применить и настроить/оптимизировать CatBoost для решения рассматриваемой задачи классификации.

Задание 1.

Начнем с загрузки и предварительной обработки данных:

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("archive/creditcard.csv")

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

Теперь приступим к построению моделей.

Бэггинг:

In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
bagging.fit(X_train, y_train)

y_pred = bagging.predict(X_test)
f1_score(y_test, y_pred, average="macro")

0.9193496436735217

Случайный лес:

In [4]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
f1_score(y_test, y_pred, average="macro")

0.9273695600665434

LightGBM:

In [5]:
%pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting_type": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1,
    "random_state": 42
}

num_rounds = 100
lgb_model = lgb.train(params, lgb_train, num_boost_round=num_rounds, valid_sets=lgb_test)

y_pred = lgb_model.predict(X_test) > 0.5
f1_score(y_test, y_pred, average="macro")


[1]	valid_0's binary_logloss: 0.00777262
[2]	valid_0's binary_logloss: 0.021733
[3]	valid_0's binary_logloss: 0.0164122
[4]	valid_0's binary_logloss: 0.0158012
[5]	valid_0's binary_logloss: 0.0183811
[6]	valid_0's binary_logloss: 0.0176908
[7]	valid_0's binary_logloss: 0.0184361
[8]	valid_0's binary_logloss: 0.0176345
[9]	valid_0's binary_logloss: 0.0156961
[10]	valid_0's binary_logloss: 0.0173275
[11]	valid_0's binary_logloss: 0.0184398
[12]	valid_0's binary_logloss: 0.0205078
[13]	valid_0's binary_logloss: 0.0241834
[14]	valid_0's binary_logloss: 0.0216466
[15]	valid_0's binary_logloss: 0.0216203
[16]	valid_0's binary_logloss: 0.0217693
[17]	valid_0's binary_logloss: 0.0219467
[18]	valid_0's binary_logloss: 0.0218667
[19]	valid_0's binary_logloss: 0.0239672
[20]	valid_0's binary_logloss: 0.0257321
[21]	valid_0's binary_logloss: 0.0256758
[22]	valid_0's binary_logloss: 0.0245264
[23]	valid_0's binary_logloss: 0.0244163
[24]	valid_0's binary_logloss: 0.0243758
[25]	valid_0's binary_log

0.6861004280827407

Задание 2.

Для оптимизации моделей можно использовать GridSearchCV или RandomizedSearchCV. Например, для случайного леса:

In [8]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, None],
    "max_features": ["sqrt", "log2", None]
}

rf = RandomForestClassifier(random_state=42)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=10, cv=3, random_state=42)
rf_random.fit(X_train, y_train)

best_rf = rf_random.best_estimator_
y_pred = best_rf.predict(X_test)
f1_score(y_test, y_pred, average="macro")


KeyboardInterrupt: 

Матрица ошибок для лучшей модели:

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


Задание 3.

Для применения и оптимизации CatBoost нам необходимо установить библиотеку:

In [ ]:
%pip install catboost

После этого можно построить модель:

In [ ]:
from catboost import CatBoostClassifier